In [5]:
%load_ext autoreload
%autoreload 2
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/luca/uni/master/dl-in-vc/voxl3r'

In [6]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [7]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('trame')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig

from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [8]:
import numpy as np
max_seq_len = 16
dataset = SceneDataset(
        data_dir="/home/luca/mnt/data/scannetpp/data",
        camera="iphone",
        n_points=300000,
        threshold_occ=0.01,
        representation="occ",
        visualize=True,
        max_seq_len=max_seq_len,
        resolution=0.02,
    )

device = get_default_device()
transform = SceneDatasetTransformToTorch(target_device=device)
idx = dataset.get_index_from_scene("0cf2e9402d")
chunks = dataset.chunk_whole_scene(idx, visualize=True)
data = chunks[8]

## Scene Chunking

In [9]:
from dataset import plot_occupency_grid


coordinates, occupancy_values = dataset.create_voxel_grid(idx)
plot_occupency_grid({
  "training_data" : (coordinates, occupancy_values),
  "mesh" : dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply",
})

Voxel Grid Size (L, W, H): (np.float64(4.66), np.float64(2.7800000000000002), np.float64(2.6200000000000006))
Voxel Grid Center: [2.29 1.35 1.27]
Occupied Voxels: 280199


Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada7164ab70_0&reconnect=auto" class="pyvi…

Exception raised
KeyError('c8bb90dd40d372ac5325cbab160019fc_105552d')
Traceback (most recent call last):
  File "/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/wslink/protocol.py", line 313, in onCompleteMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 35, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: 'c8bb90dd40d372ac5325cbab160019fc_105552d'

Exception raised
KeyError('7868f6cf000339d7210454010c276efb_177350L')
Traceback (most recent call la

In [10]:
import trimesh

mesh = trimesh.load(dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply")
boxes = mesh.voxelized(0.02).as_boxes()
visualize_mesh(mesh.voxelized(0.02).as_boxes(), opacity=1)

Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada7125ae70_1&reconnect=auto" class="pyvi…

In [11]:
visualize_mesh(
        dataset.data_dir / dataset.scenes[idx] / "scans" / "mesh_aligned_0.05.ply"

    )

Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada712f8f20_2&reconnect=auto" class="pyvi…

In [12]:
image_names, camera_params_list, camera_center = data["images"]

visualize_mesh(
        data["mesh"],
        images=image_names,
        camera_params_list=camera_params_list,
        point_coords=camera_center,
    )

Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada712fb7a0_3&reconnect=auto" class="pyvi…

## Occlupancy Computation

## Visualization: Transform to Voxel Grid

In [13]:
from dataset import plot_occupency_grid

plot_occupency_grid(data, resolution=dataset.resolution / 1.1)

Voxel Grid Size (L, W, H): (np.float64(2.552727272727274), np.float64(2.0727272727272745), np.float64(1.8527272727272743))
Voxel Grid Center: [3.34 1.   0.89]
Occupied Voxels: 21668


Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada70f0af30_4&reconnect=auto" class="pyvi…

## Project RGB Values to Scene

In [14]:
from models.surface_net_baseline.train import visualize_unprojection

visualize_unprojection(data)

/home/luca/uni/master/dl-in-vc/voxl3r/dataset.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ]
/home/luca/uni/master/dl-in-vc/voxl3r/dataset.py:427: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ).to(self.target_device)
/home/luca/uni/master/dl-in-vc/voxl3r/models/surface_net_baseline/data.py:103: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at ../aten/src/ATen/Context.cpp:296.)
  transformed_points = torch.matmul(transformations, points)
/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affi

Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada71687a70_5&reconnect=auto" class="pyvi…

## First Training Run Naive Approach

In [15]:
from models.surface_net_baseline.data import OccSurfaceNetDatamodule


datamodule = OccSurfaceNetDatamodule(
        dataset,
        ["0cf2e9402d", "0cf2e9402d", "0cf2e9402d"],
        batch_size=1,
        max_sequence_length=max_seq_len,
        single_chunk=False,
    )

In [16]:
# no pos enc w8mnl00u
# pos first 02th0eov
from utils.visualize import plot_voxel_grid


dict = torch.load(
        "./.lightning/occ-surface-net/surface-net-baseline/lpacvi7v/best_ckpts.pt"
    )
model_path = dict["best_model_val_accuracy"]

# trainer.test(model_path, dataloaders=datamodule)
datamodule.prepare_data()
datamodule.setup("test")

model = OccSurfaceNet.load_from_checkpoint(model_path)
test_dict = model.test_visualize(datamodule.test_dataloader())

gt = torch.cat(test_dict["gt"])
points = torch.cat(test_dict["points"])
y = torch.sigmoid(torch.cat(test_dict["out"]))
y[y < 0.5] = 0.0
y[y > 0.5] = 1.0

mesh = (
    Path(dataset.data_dir)
    / dataset.scenes[datamodule.scene_idx_test]
    / "scans"
    / "mesh_aligned_0.05.ply"
)
mesh = None
# visualize_mesh(mesh, point_coords=points.cpu().numpy(), heat_values=y.cpu().numpy())
plot_voxel_grid(
    points.detach().cpu().numpy(),
    y.detach().cpu().numpy(),
    ref_mesh=mesh,
    resolution=dataset.resolution,
    )


/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/torch/utils/data/dataset.py:473: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(
/home/luca/.cache/pypoetry/virtualenvs/voxl3r-xXnK9ReQ-py3.12/lib/python3.12/site-packages/torch/utils/data/dataset.py:473: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(
/home/luca/uni/master/dl-in-vc/voxl3r/dataset.py:426: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ]
/home/luca/uni/master/dl-in-vc/voxl3r/dataset.py:427: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ).to(self.target_device)
/home/luca/.cache

Voxel Grid Size (L, W, H): (np.float32(2.56), np.float32(2.08), np.float32(1.86))
Voxel Grid Center: [3.34 1.   0.89]
Occupied Voxels: 22317.0


Widget(value='<iframe src="http://localhost:46177/index.html?ui=P_0x7ada714789e0_6&reconnect=auto" class="pyvi…

In [17]:
from models.surface_net_baseline.data import OccSurfaceNetDatamodule
from utils.visualize import plot_voxel_grid

max_seq_len = 10
scene_dataset = SceneDataset(data_dir="datasets/scannetpp/data", camera="iphone", n_points=300000, threshold_occ=0.01, representation="occ", visualize=True, max_seq_len=max_seq_len)
scene_dataset.seed = 42
model_path = ".lightning/occ-surface-net/surface-net-baseline/r3cf6z5t/checkpoints/epoch=262-step=24459.00-val_accuracy=0.89.ckpt"

model = OccSurfaceNet.load_from_checkpoint(model_path)
datamodule = OccSurfaceNetDatamodule(scene_dataset, "8b2c0938d6", batch_size=128, max_sequence_length=max_seq_len)
datamodule.setup('test')
test_dict = model.test_visualize(datamodule.test_dataloader())

gt = torch.cat(test_dict['gt'])
points = torch.cat(test_dict['points'])
y = torch.sigmoid(torch.cat(test_dict['out']))
#y[y < 0.5] = 0.0
#y[y > 0.5] = 1.0

mesh = Path(scene_dataset.data_dir) / scene_dataset.scenes[datamodule.scene_idx] / "scans" / "mesh_aligned_0.05.ply"
visualize_mesh(mesh, point_coords=points.detach().cpu().numpy(), heat_values=y.detach().cpu().numpy())
#plot_voxel_grid(points.detach().cpu().numpy(), y.detach().cpu().numpy(), ref_mesh=mesh)

FileNotFoundError: [Errno 2] No such file or directory: '/home/luca/uni/master/dl-in-vc/voxl3r/.lightning/occ-surface-net/surface-net-baseline/r3cf6z5t/checkpoints/epoch=262-step=24459.00-val_accuracy=0.89.ckpt'

In [ ]:
occ = y.clone()
occ[y < 0.5] = 0.0
occ[y > 0.5] = 1.0
plot_voxel_grid(points.detach().cpu().numpy(), occ.detach().cpu().numpy(), ref_mesh=mesh)

Voxel Grid Size (L, W, H): (np.float32(2.36118), np.float32(2.178124), np.float32(1.2722685))
Voxel Grid Center: [3.4625769 2.1568832 1.4368578]
Occupied Voxels: 13377.0


Widget(value='<iframe src="http://localhost:65377/index.html?ui=P_0x2b1c78c50_5&reconnect=auto" class="pyvista…